<a href="https://colab.research.google.com/github/akshaya-nagarajan/AdvancedDeepLearning/blob/master/Assignment_4/Multitask_Regression_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required Packages

In [ ]:
!pip install autokeras

     |████████████████████████████████| 143kB 10.8MB/s 
     |████████████████████████████████| 849kB 21.0MB/s 
     |████████████████████████████████| 358kB 48.2MB/s 
     |████████████████████████████████| 174kB 18.9MB/s 
     |████████████████████████████████| 36.7MB 72kB/s 
     |████████████████████████████████| 102kB 13.6MB/s 
     |████████████████████████████████| 1.1MB 43.9MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20071 sha256=68e12614bdf53aa125b3a299ec6fa1746091cc73c9b76bebec8f36c3898d2e7d
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built py-cpuinfo


In [ ]:
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc3

  Cloning https://github.com/keras-team/keras-tuner.git (to revision 1.0.2rc3) to /tmp/pip-req-build-jf48762y
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-jf48762y
  Running command git checkout -q 172be42f1310cdbad48941494f84d65367de4f11
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2rc3-cp36-none-any.whl size=91635 sha256=a8d69bbefec515c41f2624994cd0897cbdd18ad5acbf65665cb151ac708fabc7
  Stored in directory: /tmp/pip-ephem-wheel-cache-bqw9y4pp/wheels/2d/b8/84/5f9abbcad32fc56a0fdd5d1b8c267221abb5ec582dde4371f7
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=cf7dfc9189588ccf56ad821364d073ad21e4fc1d1742ddc3de0af116902b0041
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


## Import packages, create image and structured data

In [ ]:
import numpy as np

num_instances = 100
# Generate image data.
image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
# Generate structured data.
structured_data = np.random.rand(num_instances, 20).astype(np.float32)


In [ ]:
# Generate regression targets.
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
# Generate classification labels of five classes.
classification_target = np.random.randint(5, size=num_instances)


## Train the model

In [ ]:
import autokeras as ak
# Initialize the multi with multiple inputs and outputs.
model = ak.AutoModel(
    inputs=[ak.ImageInput(), ak.StructuredDataInput()],
    outputs=[
        ak.RegressionHead(metrics=['mae']),
        ak.ClassificationHead(loss='categorical_crossentropy', metrics=['accuracy'])
    ],
    overwrite=True,
    max_trials=2)
# Fit the model with prepared data.
model.fit(
    [image_data, structured_data],
    [regression_target, classification_target],
    epochs=3)


Trial 2 Complete [00h 00m 14s]
val_loss: 1.8851583003997803

Best val_loss So Far: 1.8851583003997803
Total elapsed time: 00h 00m 36s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
4/4 [==============================] - 0s 113ms/step - loss: 3.2560 - regression_head_1_loss: 1.2879 - classification_head_1_loss: 1.9681 - regression_head_1_mae: 0.8556 - classification_head_1_accuracy: 0.1200
Epoch 2/3
4/4 [==============================] - 0s 28ms/step - loss: 3.4270 - regression_head_1_loss: 1.7754 - classification_head_1_loss: 1.6516 - regression_head_1_mae: 0.9469 - classification_head_1_accuracy: 0.3100
Epoch 3/3
4/4 [==============================] - 0s 29ms/step - loss: 5.3546 - regression_head_1_loss: 3.8231 - classification_head_1_loss: 1.5314 - regression_head_1_mae: 1.0976 - classification_head_1_accuracy: 0.4100
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should no

In [ ]:
split = 20

image_val = image_data[split:]
structured_val = structured_data[split:]
regression_val = regression_target[split:]
classification_val = classification_target[split:]

image_data = image_data[:split]
structured_data = structured_data[:split]
regression_target = regression_target[:split]
classification_target = classification_target[:split]

model.fit(
    [image_data, structured_data],
    [regression_target, classification_target],
    # Use your own validation set.
    validation_data=(
        [image_val, structured_val],
        [regression_val, classification_val]),
    epochs=2)


In [ ]:
import autokeras as ak

input_node1 = ak.ImageInput()
output_node = ak.Normalization()(input_node1)
output_node = ak.ImageAugmentation()(output_node)
output_node1 = ak.ConvBlock()(output_node)
output_node2 = ak.ResNetBlock(version='v2')(output_node)
output_node1 = ak.Merge()([output_node1, output_node2])

input_node2 = ak.StructuredDataInput()
output_node = ak.CategoricalToNumerical()(input_node2)
output_node2 = ak.DenseBlock()(output_node)

output_node = ak.Merge()([output_node1, output_node2])
output_node1 = ak.ClassificationHead()(output_node)
output_node2 = ak.RegressionHead()(output_node)

auto_model = ak.AutoModel(
    inputs=[input_node1, input_node2], 
    outputs=[output_node1, output_node2],
    overwrite=True,
    max_trials=2)

image_data = np.random.rand(num_instances, 32, 32, 3).astype(np.float32)
structured_data = np.random.rand(num_instances, 20).astype(np.float32)
regression_target = np.random.rand(num_instances, 1).astype(np.float32)
classification_target = np.random.randint(5, size=num_instances)

auto_model.fit(
    [image_data, structured_data],
    [classification_target, regression_target],
    batch_size=32,
    epochs=3)


Trial 2 Complete [00h 00m 14s]
val_loss: 1.7827478647232056

Best val_loss So Far: 1.6744154691696167
Total elapsed time: 00h 00m 28s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
4/4 [==============================] - 1s 147ms/step - loss: 3.6500 - classification_head_1_loss: 1.9526 - regression_head_1_loss: 1.6974 - classification_head_1_accuracy: 0.1400 - regression_head_1_mean_squared_error: 1.6974
Epoch 2/3
4/4 [==============================] - 0s 34ms/step - loss: 3.4357 - classification_head_1_loss: 1.4887 - regression_head_1_loss: 1.9470 - classification_head_1_accuracy: 0.3700 - regression_head_1_mean_squared_error: 1.9470
Epoch 3/3
4/4 [==============================] - 0s 32ms/step - loss: 5.0010 - classification_head_1_loss: 1.4466 - regression_head_1_loss: 3.5544 - classification_head_1_accuracy: 0.4100 - regression_head_1_mean_squared_error: 3.5544
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
